In [1]:
import pandas as pd
import numpy as np
import re
import yaml
from xml.etree import ElementTree as ET
from Bio import Entrez
from typing import List, Dict
from dotenv import load_dotenv
import os
from openai import OpenAI
import pandas as pd
import numpy as np
import re, os, glob
from typing import Dict, List, Tuple, Optional
from tqdm import tqdm
import os, time, textwrap



In [2]:

# load .env file
load_dotenv()
def load_config(yaml_path="P2-config.yaml"):
    with open(yaml_path, "r", encoding="utf-8") as f:
        return yaml.safe_load(f)
    
config = load_config()
Entrez.email = os.getenv("NCBI_EMAIL", "your_email@example.com")

api_key = os.getenv("NCBI_API_KEY")

# sanity-check
print("Key loaded:", os.getenv("OPENAI_API_KEY") is not None)

# initialize client
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.getenv("OPENAI_API_KEY"),
)

Key loaded: True


### get abstracts

In [ ]:
def normalize_pmids(series: pd.Series) -> List[str]:
    """
    Accepts a column that may contain:
      - ints
      - strings like '12345, 67890'
      - lists (already split)
    Returns a clean, unique, ordered list of PMIDs (as strings).
    """
    seen = set()
    ordered = []
    for v in series.dropna().tolist():
        # If it already looks like a list, iterate; else treat as one string
        items = v if isinstance(v, list) else re.split(r"[,\s]+", str(v))
        for p in items:
            s = p.strip()
            if not s:
                continue
            # keep only digits
            s = re.sub(r"\D", "", s)
            if s and s not in seen:
                seen.add(s)
                ordered.append(s)
    return ordered

def write_pmids_per_drug(df: pd.DataFrame, pmid_col: str = "Filtered PMIDs", out_dir: str = "abstracts"):
    os.makedirs(out_dir, exist_ok=True)
    for drug, g in df.groupby("Drug", dropna=False):
        pmids = normalize_pmids(g[pmid_col])
        fname = f"pmids_{str(drug).replace(' ', '_').lower()}.txt"
        path = os.path.join(out_dir, fname)
        with open(path, "w", encoding="utf-8") as f:
            for pmid in pmids:
                f.write(pmid + "\n")

def _parse_pubmed_xml(xml_text: str) -> List[Dict]:
    """
    Minimal PubMed XML parser: pulls PMID, Title, Abstract (joined), and
    tries to separate Results/Conclusions when section labels exist.
    """
    rows = []
    root = ET.fromstring(xml_text)

    # PubmedArticle elements
    for art in root.findall(".//PubmedArticle"):
        # PMID
        pmid_el = art.find(".//MedlineCitation/PMID")
        pmid = pmid_el.text.strip() if pmid_el is not None and pmid_el.text else ""

        # Title
        title_el = art.find(".//Article/ArticleTitle")
        title = "".join(title_el.itertext()).strip() if title_el is not None else ""

        # Abstract: may have multiple AbstractText nodes, possibly with Label attrs
        abstract_nodes = art.findall(".//Article/Abstract/AbstractText")
        parts = []
        results = []
        conclusions = []
        for node in abstract_nodes:
            label = (node.get("Label") or node.get("NlmCategory") or "").strip().lower()
            text = "".join(node.itertext()).strip()
            if not text:
                continue
            parts.append(text)
            if "result" in label:
                results.append(text)
            if "conclusion" in label:
                conclusions.append(text)

        abstract = "\n".join(parts).strip()
        results_txt = "\n".join(results).strip()
        conclusions_txt = "\n".join(conclusions).strip()

        rows.append({
            "PMID": pmid,
            "Title": title,
            "Abstract": abstract,
            "Results": results_txt,
            "Conclusions": conclusions_txt
        })
    return rows

def fetch_pubmed_abstracts(pmids: List[str],
                           batch_size: int = 200,
                           sleep_between: float = 0.5,
                           max_retries: int = 3) -> pd.DataFrame:
    """
    Fetch PubMed records for given PMIDs.
    - Dedupes while preserving order.
    - Retries with exponential backoff.
    - Respects a delay between calls (sleep_between).
    """
    # clean/dedupe while preserving order
    seen, ordered_pmids = set(), []
    for p in pmids:
        s = re.sub(r"\D", "", str(p)).strip()
        if s and s not in seen:
            seen.add(s)
            ordered_pmids.append(s)

    rows: List[Dict] = []
    for i in range(0, len(ordered_pmids), batch_size):
        chunk = ordered_pmids[i:i+batch_size]
        ids = ",".join(chunk)

        for attempt in range(1, max_retries + 1):
            try:
                with Entrez.efetch(db="pubmed", id=ids, retmode="xml") as handle:
                    xml_text = handle.read()
                rows.extend(_parse_pubmed_xml(xml_text))
                break
            except Exception as e:
                if attempt == max_retries:
                    # fill with empty rows so downstream code can keep order
                    rows.extend([{"PMID": p, "Title": "", "Abstract": } for p in chunk])
                    print(f"[WARN] Failed batch starting {chunk[0]}: {e}")
                else:
                    wait = sleep_between * (2 ** (attempt - 1))
                    print(f"[INFO] Retry {attempt}/{max_retries} after error: {e}. Sleeping {wait:.2f}s")
                    time.sleep(wait)
        time.sleep(sleep_between)

    df_out = pd.DataFrame(rows).drop_duplicates(subset=["PMID"]).reset_index(drop=True)

    # restore original order
    order_map = {p: idx for idx, p in enumerate(ordered_pmids)}
    df_out["__order"] = df_out["PMID"].map(order_map)
    df_out = df_out.sort_values("__order").drop(columns="__order").reset_index(drop=True)
    return df_out

def save_abstracts_per_drug(df_drugs: pd.DataFrame,
                            abstracts_df: pd.DataFrame,
                            out_dir):

    os.makedirs(out_dir, exist_ok=True)

    for drug, g in df_drugs.groupby("Drug"):
        pmids = normalize_pmids(g["PMIDs"])
        fname = f"{drug.replace(' ','_').lower()}_abstracts.txt"
        path  = os.path.join(out_dir, fname)

        with open(path, "w", encoding="utf-8") as f:
            for p in pmids:
                row = abstracts_df[abstracts_df["PMID"] == p]
                if row.empty:
                    continue
                abs_txt = (row.iloc[0]["Abstract"] or "").strip()

                f.write(f"pmid: {p}\n")
                f.write(abs_txt)
                f.write("\n\n")  # spacing between abstracts

        print(f"Wrote {path}")

In [ ]:


for pair_cfg in config.get("disease_pairs", []):
    pair_name = pair_cfg.get("name", "unnamed_pair")
    print(f"\n=== Running disease pair: {pair_name} ===")
    Entrez.tool = pair_cfg['abstract_summary']['entrez_tool']
    drug_abstracts = pair_cfg['abstract_summary']['drug_abstract']
    df = pd.read_csv(pair_cfg['initial_pmid_list'])
    df = df[
            df["Drug"].str.lower() != "epidermal growth factor"
        ].reset_index(drop=True)
    
    ## clear out_dir
    
    write_pmids_per_drug(df, pmid_col="PMIDs", out_dir=drug_abstracts)
    all_pmids = normalize_pmids(df["PMIDs"])
    abstracts_df = fetch_pubmed_abstracts(all_pmids, batch_size=200, sleep_between=0.5, max_retries=3)
    abstracts_df = fetch_pubmed_abstracts(all_pmids)
    save_abstracts_per_drug(df, abstracts_df, out_dir=drug_abstracts)
    




=== Running disease pair: diabetes_crc ===
Wrote C:/Users/Admin/OneDrive - Nanyang Technological University/FYP_DATA/LITERATURE/drug_abstracts\acetaminophen_abstracts.txt
Wrote C:/Users/Admin/OneDrive - Nanyang Technological University/FYP_DATA/LITERATURE/drug_abstracts\adenosine_monophosphate_abstracts.txt
Wrote C:/Users/Admin/OneDrive - Nanyang Technological University/FYP_DATA/LITERATURE/drug_abstracts\adenosine_triphosphate_abstracts.txt
Wrote C:/Users/Admin/OneDrive - Nanyang Technological University/FYP_DATA/LITERATURE/drug_abstracts\albiglutide_abstracts.txt
Wrote C:/Users/Admin/OneDrive - Nanyang Technological University/FYP_DATA/LITERATURE/drug_abstracts\allopurinol_abstracts.txt
Wrote C:/Users/Admin/OneDrive - Nanyang Technological University/FYP_DATA/LITERATURE/drug_abstracts\alogliptin_abstracts.txt
Wrote C:/Users/Admin/OneDrive - Nanyang Technological University/FYP_DATA/LITERATURE/drug_abstracts\amphetamine_abstracts.txt
Wrote C:/Users/Admin/OneDrive - Nanyang Technologi

### summarise abstracts

In [8]:
def _norm(s: str) -> str:
    return re.sub(r"\s+", " ", str(s).strip().lower())

def _read_text_file(path: str) -> Optional[str]:
    try:
        with open(path, "r", encoding="utf-8") as f:
            txt = f.read().strip()
            return txt if txt else None
    except Exception:
        return None
    
def parse_drug_abstract_file(txt: str) -> List[Tuple[str, str]]:
    """
    Robustly parse blocks like:
        pmid: 12345
        abstract line 1
        abstract line 2
    """
    txt = txt.replace("\r", "")
    lines = txt.strip().split("\n")

    entries = []
    current_pmid = None
    current_lines = []

    for line in lines:
        line_strip = line.strip()
        if not line_strip:
            continue  # skip empty lines

        # detect new PMID line
        m = re.match(r"pmid:\s*(\d+)", line_strip, flags=re.IGNORECASE)
        if m:
            # save previous abstract if any
            if current_pmid and current_lines:
                entries.append((current_pmid, " ".join(current_lines).strip()))
            # start new block
            current_pmid = m.group(1)
            current_lines = []
        else:
            # regular abstract text line
            current_lines.append(line_strip)

    # append last one
    if current_pmid and current_lines:
        entries.append((current_pmid, " ".join(current_lines).strip()))

    return entries


def load_reports_dir(
    reports_dirs: List[str]
) -> Tuple[Dict[str, str], Dict[str, List[Tuple[str, str]]]]:

    # two patterns: new and legacy
    pat_new    = re.compile(r"^(.+?)_abstracts(?:_\d{8}_\d{6})?\.txt$", re.IGNORECASE)


    raw_text_by_drug: Dict[str, str] = {}
    parsed_by_drug:  Dict[str, List[Tuple[str, str]]] = {}

    for d in reports_dirs:
        if not os.path.isdir(d):
            continue
        for p in glob.glob(os.path.join(d, "*.txt")):
            fn = os.path.basename(p)
            m = pat_new.match(fn) 
            if not m:
                continue
            drug_norm = _norm(m.group(1))
            txt = _read_text_file(p)
            if not txt:
                continue
            # keep the most recent if multiple match; simple length heuristic
            if drug_norm in raw_text_by_drug and len(txt) <= len(raw_text_by_drug[drug_norm]):
                continue
            raw_text_by_drug[drug_norm] = txt
            parsed_by_drug[drug_norm]   = parse_drug_abstract_file(txt)

    return raw_text_by_drug, parsed_by_drug

In [9]:
def _extract_text_from_response(resp) -> str:
    # works across SDK variants
    txt = ""
    try:
        txt = (getattr(resp, "output_text", "") or "").strip()
    except Exception:
        pass
    if not txt:
        try:
            parts = []
            for item in getattr(resp, "output", []) or []:
                for c in getattr(item, "content", []) or []:
                    # some SDKs expose .text.value
                    v = getattr(c, "text", None)
                    if isinstance(v, dict):
                        parts.append(v.get("value", ""))
                    elif isinstance(v, str):
                        parts.append(v)
            txt = "\n".join([p for p in parts if p]).strip()
        except Exception:
            pass
    return txt

def summarise_abstract(drug: str, pmid: str, abstract: str, retries: int = 3) -> str:
    """
    Summarise how {drug} relates to colorectal cancer (CRC) and/or diabetes,
    including mechanistic or pathway-level information even if no direct disease link is stated.
    Allow inferential interpretation (e.g., "may influence", "supported by cases"),
    but avoid speculation beyond what the abstract supports.
    Adaptive retry: raise max_output_tokens, then truncate input if still incomplete.
    If the abstract is irrelevant to both CRC and diabetes, exclude it from output entirely.
    """
    base_prompt = f"""
    You are an expert biomedical summariser.
    From the following abstract, describe how {drug} relates to colorectal cancer (CRC)
    and/or diabetes, including mechanistic or biological pathway details that could
    help explain potential associations, even if not explicitly tied to CRC or diabetes.
    If the abstract is irrelevant to CRC and diabetes, output exactly: "No relevant information found."
    You may infer relationships when the evidence is contextually supported
    (e.g., "suggests involvement", "supported by experimental findings"),
    You may phrase findings as inferential (e.g., "supported by cases", "suggests an association")
    if warranted by the text.
    but do not introduce new facts or broad speculation.

    Write up to 2 sentences (maximum 100 words).

    Abstract (PMID: {pmid}):
    {{abstract_block}}
    """.strip()

    # progressive settings
    attempts = [
        {"max_output_tokens": 320, "truncate_chars": None},
        {"max_output_tokens": 640, "truncate_chars": None},
        {"max_output_tokens": 640, "truncate_chars": 1800},
    ]

    for i, cfg in enumerate(attempts, start=1):
        a = abstract if not cfg["truncate_chars"] else abstract[:cfg["truncate_chars"]]
        prompt = base_prompt.replace("{abstract_block}", a)

        try:
            resp = client.responses.create(
                model="gpt-5-mini",
                input=prompt,
                max_output_tokens=cfg["max_output_tokens"],
                reasoning={"effort": "low"},  # keep reasoning token use modest
            )

            # If API signals incomplete due to token cap, escalate
            if getattr(resp, "status", "") == "incomplete":
                reason = getattr(getattr(resp, "incomplete_details", None), "reason", "")
                if reason == "max_output_tokens":
                    # escalate to next attempt
                    continue

            out = _extract_text_from_response(resp).strip()
            if not out:
                # empty output: escalate
                continue
            return out

        except Exception as e:
            print(f"[WARN] {drug} PMID {pmid} attempt {i} failed: {e}")
            time.sleep(2 ** (i-1))

    return "No relevant information found."

In [10]:
for pair_cfg in config.get("disease_pairs", []):
    pair_name = pair_cfg.get("name", "unnamed_pair")
    print(f"\n=== Running disease pair: {pair_name} ===")
    abstract_reports_path = pair_cfg['abstract_summary']['drug_abstract']

    raw_text_by_drug, parsed_by_drug = load_reports_dir(
        [abstract_reports_path]
    )

    output_dir = pair_cfg['abstract_summary']['summarised_abstracts']
    os.makedirs(output_dir, exist_ok=True)
    for drug, entries in parsed_by_drug.items():
        drug_clean = drug.replace(" ", "_").lower()
        out_path = os.path.join(output_dir, f"{drug_clean}_summary.txt")

        with open(out_path, "w", encoding="utf-8") as f_out:
            for pmid, abstract in tqdm(entries, desc=f"Summarising {drug}", unit="abs"):
                print("TEST:", drug, pmid)

                summary = summarise_abstract(drug, pmid, abstract)
                wrapped = textwrap.fill(summary, width=100, replace_whitespace=False)

                f_out.write(f"PMID: {pmid}\n")
                f_out.write(wrapped + "\n\n")



=== Running disease pair: diabetes_crc ===


Summarising acetaminophen:   0%|          | 0/1 [00:00<?, ?abs/s]

TEST: acetaminophen 35839363


Summarising adenosine_monophosphate:   0%|          | 0/3 [00:00<?, ?abs/s]

TEST: adenosine_monophosphate 37071615


...

Summarising vitamin_b6:   0%|          | 0/1 [00:00<?, ?abs/s]

TEST: vitamin_b6 36367997


Summarising zanubrutinib:   0%|          | 0/1 [00:00<?, ?abs/s]

TEST: zanubrutinib 34886775


Summarising zanubrutinib: 100%|██████████| 1/1 [00:05<00:00,  5.25s/abs]
